<a href="https://colab.research.google.com/github/DevSingh28/Text_Generation/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Libraries and Modules Used:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00


In [ ]:
# Loading Dataset

from datasets import load_dataset

dataset = load_dataset("daily_dialog")
data = dataset['train'].shuffle(seed=42).select(range(int(len(dataset['train']) * 0.1)))

for i in range(5):
    print(data[i])

data = ' '.join([' '.join(dialogue['dialog']) for dialogue in data])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

daily_dialog.py:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'dialog': ["finally we're on board ! ", " yes . It was so crowded . I'm worn-out . Let's find our seats . ", ' are they window seats or aisle seats ? ', ' let me see ... yes , one window seat and one aisle seat . ', " ok . But can I trade my seat with you ? I prefer the one near the window . I'm a terrible flyer . I always get airsick and can never relax until after I've landed . ", " that's fine . I'd like to be on the aisle anyway . It's easier to get in and out . ", ' thanks . Where shall we put our luggage ? ', ' I think the smaller carry-on bag can go in the overhead compartment , and the others can go under the seat . ', ' good idea . ', " don't forget to keep the seat belt on . ", " ok . Hope it's a pleasant trip . ", ' yes ! ', ' and no hijackers . ', ' oh , you have too wild of an imagination . '], 'act': [1, 3, 2, 1, 3, 4, 2, 1, 1, 3, 4, 1, 1, 1], 'emotion': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'dialog': ["Mary , why don't you come to my house this Saturday ? ", " I 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# Tokenize the data

chars = sorted(list(set(data)))
vocab_size = len(chars)
print(vocab_size)
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

87


In [ ]:
chars

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '\\',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '£',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '。']

In [ ]:
# Hyperparameters

embedding_size = 64
num_heads = 4
num_layers = 2
hidden_size = 256
seq_length = 12
batch_size = 64
learning_rate = 0.0005
num_epochs = 15
vocab_size = 87

In [ ]:
# Convert data into input-output pairs for training

data_idx = [char_to_idx[ch] for ch in data]
inputs = []
targets = []

for i in range(0, len(data_idx) - seq_length):
    inputs.append(data_idx[i:i + seq_length])
    targets.append(data_idx[i + 1:i + seq_length + 1])

inputs = torch.tensor(inputs, dtype=torch.long).to(device)
targets = torch.tensor(targets, dtype=torch.long).to(device)

In [ ]:
# Loading data and using DataLoader for better performance

dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# GPT-like model using Transformer architecture

class GPTModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_heads, hidden_size, num_layers, seq_length):
        super(GPTModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.position_embedding = nn.Embedding(seq_length, embedding_size)
        self.transformer_layers = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=num_heads, dim_feedforward=hidden_size, batch_first=True),
            num_layers=num_layers
        )
        self.fc_out = nn.Linear(embedding_size, vocab_size)

    def forward(self, x):
        seq_length = x.size(1)
        positions = torch.arange(0, seq_length, device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.position_embedding(positions)
        x = self.transformer_layers(x)
        x = self.fc_out(x)
        return x

In [ ]:
# Instantiate the model

model = GPTModel(vocab_size, embedding_size, num_heads, hidden_size, num_layers, seq_length).to(device)

In [ ]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
print(len(inputs) // batch_size)

8495


In [ ]:
# Training loop with batching

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    with tqdm(total=len(dataloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for batch_inputs, batch_targets in dataloader:
            batch_inputs, batch_targets = batch_inputs.to(device), batch_targets.to(device)  # Move to GPU
            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs.view(-1, vocab_size), batch_targets.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            pbar.update(1)

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}')

Epoch 1/15:   0%|          | 0/8496 [00:00<?, ?batch/s]

Epoch 1, Average Loss: 0.2231


Epoch 2/15:   0%|          | 0/8496 [00:00<?, ?batch/s]

Epoch 2, Average Loss: 0.1462


Epoch 3/15:   0%|          | 0/8496 [00:00<?, ?batch/s]

...

Epoch 13, Average Loss: 0.1260


Epoch 14/15:   0%|          | 0/8496 [00:00<?, ?batch/s]

Epoch 14, Average Loss: 0.1256


Epoch 15/15:   0%|          | 0/8496 [00:00<?, ?batch/s]

Epoch 15, Average Loss: 0.1251


In [ ]:
# Text generation function
def generate_text(model, start_text, max_length=100, temperature=1.0):
    model.eval()
    generated = [char_to_idx[ch] for ch in start_text]

    for _ in range(max_length):
        input_seq = torch.tensor(generated[-seq_length:], dtype=torch.long).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_seq)
        output = output[:, -1, :]

        output = output / temperature
        next_token_probs = F.softmax(output, dim=-1).squeeze()
        next_token_idx = torch.multinomial(next_token_probs, 1).item()

        generated.append(next_token_idx)

    return ''.join([idx_to_char[idx] for idx in generated])

In [ ]:
# Example text generation
start_text = "Hello How are you"
print(generate_text(model, start_text=start_text, temperature=0.7))

Hello How are you ?   Yeah , I ’ m can I ’ m are .   I want to need has the more were to in the different .   Why new


In [ ]:
def save_model(model, optimizer, epoch, file_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, file_path)
    print(f'Model saved to {file_path}')

save_model(model, optimizer, num_epochs, 'gpt_model.pth')

Model saved to gpt_model.pth


In [ ]:
def load_model(model, optimizer, file_path):
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(f'Model loaded from {file_path} at epoch {epoch}')
    return epoch

loaded_epoch = load_model(model, optimizer, 'gpt_model.pth')

Model loaded from gpt_model.pth at epoch 15


<ipython-input-19-cd21664b2664>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(file_path)
